# Import packages

In [1]:
import numpy as np
import time

# PQR

In [182]:
def PQR(matrix, n, m, r = 0, eps = 0):

    if not r or r > min(m, n):
        r = min(m, n)

    Q = np.zeros((n, r))
    R = np.zeros((r, m))
    P = np.arange(m)

    tol = eps + 1
    i = 0

    while i < r and tol > eps:

        norms = np.array([np.linalg.norm(matrix[:, j]) for j in range(i, m)])
        tol = np.sqrt(np.sum(norms**2))
        args = np.argsort(norms)[::-1]
        args += i

        maxi = min(m, i + 2)
        P[i:maxi] = P[args[:2]]
        R[:, i:maxi] = R[:, args[:2]]
        matrix[:, i:maxi] = matrix[:, args[:2]]

        R[i, i] = norms[args[0] - i]
        Q[:, i] = matrix[:, i] / R[i, i]

        for j in range(i + 1, m):
            R[i, j] = matrix[:, j] @ Q[:, i]
            matrix[:, j] -= R[i, j] * Q[:, i]

        i += 1

    return Q, R, P

# MaxVal

In [143]:
def maxval(mat, n, m, r, eps = 0):

    if not r or r > min(n, m):
          r = min(n, m)

    rows = np.random.permutation(n)
    cols = np.random.permutation(m)

    while np.linalg.det(mat[np.ix_(rows[:r], cols[:r])]) == 0:
        rows = np.random.permutation(n)
        cols = np.random.permutation(m)

    matrix = mat[:, cols]

    C = matrix[:, :r]
    A = C[:r, :]
    inv_A = np.linalg.inv(A)

    prod = C @ inv_A

    max_val_ind = np.unravel_index(np.argmax(np.abs(prod), axis=None), prod.shape)

    iter = 0

    R = np.arange(r)

    while max_val_ind[0] >= r and iter < r * r:

        iter += 1

        vec = prod[max_val_ind[0], :].copy()
        vec[max_val_ind[1]] -= 1
        vec /= prod[max_val_ind]

        prod -= prod[:, max_val_ind[1]].reshape((-1, 1)) @ vec.reshape((1, -1))

        R[max_val_ind[1]] = max_val_ind[0]

        max_val_ind = np.unravel_index(np.argmax(np.abs(prod), axis=None), prod.shape)

    rev_cols = np.zeros(m, dtype=int)
    for i in range(m):
        rev_cols[cols[i]] = i

    R = matrix[np.ix_(R, rev_cols)]

    return prod, R

# Cross Approximation

In [4]:
def cross_approx(matrix, n, m, r = 0, eps = 0):
    if not r and r > min(m, n):
        r = min(m, n)

    Q = np.zeros((n, r))
    R = np.zeros((r, m))

    vec = np.random.permutation(m)

    i = 0

    max_el = eps + 1

    while i < r and max_el * np.sqrt((n - i) * (m - i)) > eps:

        col = vec[i]

        row = np.argmax(np.abs(matrix[:, col]), axis=None)
        col = np.argmax(np.abs(matrix[row, :]), axis=None)
        row = np.argmax(np.abs(matrix[:, col]), axis=None)

        Q[:, i] = matrix[:, col] / matrix[row, col]
        R[i, :] = matrix[row, :]

        max_el = np.abs(matrix[row, col])

        matrix -= Q[:, i].reshape((-1, 1)) @ R[i, :].reshape((1, -1))

        i += 1

    return Q, R

# Create matrix

In [183]:
n = 100
m = 1000
mat_cr = np.zeros((n, m))

for i in range(n):
    for j in range(m):
        mat_cr[i, j] = np.sin(i + j)

In [188]:
matrix = mat_cr.copy()
mat_copy = mat_cr.copy()

# Run Cross Approximation

In [185]:
start_time = time.time()
Q, R = cross_approx(matrix, n, m, 2)
end_time = time.time()
print(f"time = {end_time - start_time}")

print(np.linalg.norm(Q @ R - mat_copy) / np.linalg.norm(mat_copy))

time = 0.0011477470397949219
1.0450211085339994e-16


# Run PQR

In [187]:
start_time = time.time()
Q, R, P = PQR(matrix, n, m, 2)
end_time = time.time()
print(f"time = {end_time - start_time}")

print(np.linalg.norm(Q @ R - mat_copy[:, P]) / np.linalg.norm(mat_copy))

time = 0.017113924026489258
1.6583797065978548e-16


# Run MaxVal

In [189]:
start_time = time.time()
Q, R = maxval(matrix, n, m, 2)
end_time = time.time()
print(f"time = {end_time - start_time}")

print(np.linalg.norm(Q @ R - mat_copy) / np.linalg.norm(mat_copy))

time = 0.002177715301513672
2.5134390342082333e-16
